# Data PreProcessing

# Import Data

In [45]:
import csv
import re
import nltk
import Sastrawi
import sklearn
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np

In [15]:
df = pd.read_csv("data fix\\byuidn.csv")
df = df.drop('Unnamed: 0',1)
df

,waktu,id,username,teks
0,2021-06-30 23:01:28,1410372945520652291,Dito (ディト),Ini gimana kok berhari² aplikasi gak bisa dibu...
1,2021-06-30 23:00:27,1410372691446558721,Magnet Kulkas,"Min @byu_id ini pengertian beud, makasih ya......"
2,2021-06-30 23:00:10,1410372619170324481,Anfhd,@byu_id hei masa tiap bulan begini terus.. ban...
3,2021-06-30 22:10:27,1410360109385936898,Fructose baby🌻,Ya Allah jaringan @byu_id jelek bgt udah hubun...
4,2021-06-30 22:05:43,1410358915850588179,Fructose baby🌻,@amourtabells @byu_id Paket unlimited Xl 1 bul...
...,...,...,...,...
1805,2021-06-22 15:18:45,1407357398679900169,Mengandung amarah tidak bisa dilahirkan,"Ingin jdi gamer noob? Ayo, pakai @byu_id hanya..."
1806,2021-06-22 15:06:49,1407354394551615488,Safira Dhyanti,Nanya sama mimin @byu_id gapernah di bales :(
1807,2021-06-22 15:06:49,1407354392165044225,Stand Up Comedy Pekanbaru,"@byu_id lagi ada acara keren banget ini, Impro..."
1808,2021-06-22 15:04:47,1407353883156877312,feb,knp si gangguan mulu\n\np\n@byu_id


# Cleansing Data

In [16]:
import string 
import re #regex library

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention @
    text = re.sub(r"[@][\w_-]+","", text)
    # remove link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
df['cleansing'] = df['teks'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['cleansing'] = df['cleansing'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['cleansing'] = df['cleansing'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['cleansing'] = df['cleansing'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['cleansing'] = df['cleansing'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['cleansing'] = df['cleansing'].apply(remove_singl_char)

# remove duplicate
df.drop_duplicates(subset=['cleansing'], inplace=True)

df.loc[0:9,['teks', 'cleansing']]

,teks,cleansing
0,Ini gimana kok berhari² aplikasi gak bisa dibu...,Ini gimana kok berhari aplikasi gak bisa dibuk...
1,"Min @byu_id ini pengertian beud, makasih ya......",Min ini pengertian beud makasih ya Cus nonton ...
2,@byu_id hei masa tiap bulan begini terus.. ban...,hei masa tiap bulan begini terus bantuin sini ...
3,Ya Allah jaringan @byu_id jelek bgt udah hubun...,Ya Allah jaringan jelek bgt udah hubungin CS d...
4,@amourtabells @byu_id Paket unlimited Xl 1 bul...,Paket unlimited Xl bulan kaya byu mbps yang be...
5,Alhamdulillah... Terima kasih @byu_id https://...,Alhamdulillah Terima kasih
6,"@byu_id Min tetep di 2 Mbps aja ya, jangan bal...",Min tetep di Mbps aja ya jangan balik ke Mbps ...
7,"Kirain ini prank, ternyata beneran bisa tingga...",Kirain ini prank ternyata beneran bisa tinggal...
8,Ini beneran ga min??\n@byu_id https://t.co/3NV...,Ini beneran ga min
9,"@esmrld___ @txtdrhotel @byu_id Sama bgt,halo k...",Sama bgthalo kedodoran sebelum sebulan di back...


# Case Folding

In [17]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
df['case_folding'] = df['cleansing'].str.lower()

df.loc[0:9,['cleansing', 'case_folding']]

,cleansing,case_folding
0,Ini gimana kok berhari aplikasi gak bisa dibuk...,ini gimana kok berhari aplikasi gak bisa dibuk...
1,Min ini pengertian beud makasih ya Cus nonton ...,min ini pengertian beud makasih ya cus nonton ...
2,hei masa tiap bulan begini terus bantuin sini ...,hei masa tiap bulan begini terus bantuin sini ...
3,Ya Allah jaringan jelek bgt udah hubungin CS d...,ya allah jaringan jelek bgt udah hubungin cs d...
4,Paket unlimited Xl bulan kaya byu mbps yang be...,paket unlimited xl bulan kaya byu mbps yang be...
5,Alhamdulillah Terima kasih,alhamdulillah terima kasih
6,Min tetep di Mbps aja ya jangan balik ke Mbps ...,min tetep di mbps aja ya jangan balik ke mbps ...
7,Kirain ini prank ternyata beneran bisa tinggal...,kirain ini prank ternyata beneran bisa tinggal...
8,Ini beneran ga min,ini beneran ga min
9,Sama bgthalo kedodoran sebelum sebulan di back...,sama bgthalo kedodoran sebelum sebulan di back...


# Tokenize

In [18]:
# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['tokenize'] = df['case_folding'].apply(word_tokenize_wrapper)

df.loc[0:9,['case_folding', 'tokenize']]

,case_folding,tokenize
0,ini gimana kok berhari aplikasi gak bisa dibuk...,"[ini, gimana, kok, berhari, aplikasi, gak, bis..."
1,min ini pengertian beud makasih ya cus nonton ...,"[min, ini, pengertian, beud, makasih, ya, cus,..."
2,hei masa tiap bulan begini terus bantuin sini ...,"[hei, masa, tiap, bulan, begini, terus, bantui..."
3,ya allah jaringan jelek bgt udah hubungin cs d...,"[ya, allah, jaringan, jelek, bgt, udah, hubung..."
4,paket unlimited xl bulan kaya byu mbps yang be...,"[paket, unlimited, xl, bulan, kaya, byu, mbps,..."
5,alhamdulillah terima kasih,"[alhamdulillah, terima, kasih]"
6,min tetep di mbps aja ya jangan balik ke mbps ...,"[min, tetep, di, mbps, aja, ya, jangan, balik,..."
7,kirain ini prank ternyata beneran bisa tinggal...,"[kirain, ini, prank, ternyata, beneran, bisa, ..."
8,ini beneran ga min,"[ini, beneran, ga, min]"
9,sama bgthalo kedodoran sebelum sebulan di back...,"[sama, bgthalo, kedodoran, sebelum, sebulan, d..."


# Normalisasi

In [19]:
normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df['normalisasi'] = df['tokenize'].apply(normalized_term)

df.loc[0:9,['tokenize', 'normalisasi']]

,tokenize,normalisasi
0,"[ini, gimana, kok, berhari, aplikasi, gak, bis...","[ini, bagaimana, kok, berhari, aplikasi, tidak..."
1,"[min, ini, pengertian, beud, makasih, ya, cus,...","[admin, ini, pengertian, banget, makasih, ya, ..."
2,"[hei, masa, tiap, bulan, begini, terus, bantui...","[hei, masa, tiap, bulan, begini, terus, bantui..."
3,"[ya, allah, jaringan, jelek, bgt, udah, hubung...","[ya, allah, jaringan, jelek, banget, sudah, hu..."
4,"[paket, unlimited, xl, bulan, kaya, byu, mbps,...","[paket, unlimited, xl, bulan, kaya, byu, mbps,..."
5,"[alhamdulillah, terima, kasih]","[alhamdulillah, terima, kasih]"
6,"[min, tetep, di, mbps, aja, ya, jangan, balik,...","[admin, tetap, di, mbps, aja, ya, jangan, bali..."
7,"[kirain, ini, prank, ternyata, beneran, bisa, ...","[kirain, ini, prank, ternyata, beneran, bisa, ..."
8,"[ini, beneran, ga, min]","[ini, beneran, tidak, admin]"
9,"[sama, bgthalo, kedodoran, sebelum, sebulan, d...","[sama, bgthalo, kedodoran, sebelum, sebulan, d..."


# Filtering/Stopwords

In [20]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("combined_stop_words.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    #kata = [word for word in words if word not in list_stopwords]
    return [word for word in words if word not in list_stopwords]

df['stopwords'] = df['normalisasi'].apply(stopwords_removal) 

df.loc[0:9,['normalisasi', 'stopwords']]

,normalisasi,stopwords
0,"[ini, bagaimana, kok, berhari, aplikasi, tidak...","[berhari, aplikasi, dibuka, website, diakses, ..."
1,"[admin, ini, pengertian, banget, makasih, ya, ...","[admin, pengertian, banget, makasih, cus, nonton]"
2,"[hei, masa, tiap, bulan, begini, terus, bantui...","[hei, bantuin, kirim, dm]"
3,"[ya, allah, jaringan, jelek, banget, sudah, hu...","[allah, jaringan, jelek, banget, hubungin, cs,..."
4,"[paket, unlimited, xl, bulan, kaya, byu, mbps,...","[paket, unlimited, xl, kaya, byu, mbps, bebas,..."
5,"[alhamdulillah, terima, kasih]","[alhamdulillah, terima, kasih]"
6,"[admin, tetap, di, mbps, aja, ya, jangan, bali...","[admin, mbps, mbps, gapapa, ribu, aman, yusupa..."
7,"[kirain, ini, prank, ternyata, beneran, bisa, ...","[kirain, prank, beneran, tinggal, login, pakai..."
8,"[ini, beneran, tidak, admin]","[beneran, admin]"
9,"[sama, bgthalo, kedodoran, sebelum, sebulan, d...","[bgthalo, kedodoran, sebulan, backup, byu]"


## Stemming

In [21]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['stopwords']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['stemming'] = df['stopwords'].swifter.apply(get_stemmed_term)

df.loc[0:9,['stopwords', 'stemming']]

Pandas Apply:   0%|          | 0/1729 [00:00<?, ?it/s]

,stopwords,stemming
0,"[berhari, aplikasi, dibuka, website, diakses, ...","[hari, aplikasi, buka, website, akses, cek, ku..."
1,"[admin, pengertian, banget, makasih, cus, nonton]","[admin, erti, banget, makasih, cus, nonton]"
2,"[hei, bantuin, kirim, dm]","[hei, bantuin, kirim, dm]"
3,"[allah, jaringan, jelek, banget, hubungin, cs,...","[allah, jaring, jelek, banget, hubungin, cs, d..."
4,"[paket, unlimited, xl, kaya, byu, mbps, bebas,...","[paket, unlimited, xl, kaya, byu, mbps, bebas,..."
5,"[alhamdulillah, terima, kasih]","[alhamdulillah, terima, kasih]"
6,"[admin, mbps, mbps, gapapa, ribu, aman, yusupa...","[admin, mbps, mbps, gapapa, ribu, aman, yusupa..."
7,"[kirain, prank, beneran, tinggal, login, pakai...","[kirain, prank, beneran, tinggal, login, pakai..."
8,"[beneran, admin]","[beneran, admin]"
9,"[bgthalo, kedodoran, sebulan, backup, byu]","[bgthalo, kedodoran, bulan, backup, byu]"


In [22]:
df.to_csv("data\datacleansing.csv")
df.to_excel("data\datacleansing.xlsx")